In [1]:
import pandas as pd
import os
import random
from datetime import datetime

In [1]:
import pandas as pd
import os
from datetime import datetime

def generate_all_itineraries(flight_leg_df, max_legs=5):
    sorted_df = flight_leg_df.sort_values(by='ScheduledDepartureDateTime').reset_index(drop=True)
    itinerary_id = 1
    total_rows = len(sorted_df)

    for start_idx in range(total_rows):
        current_chain = [sorted_df.iloc[start_idx]]

        for next_idx in range(start_idx + 1, min(start_idx + max_legs, total_rows)):
            prev_leg = current_chain[-1]
            next_leg = sorted_df.iloc[next_idx]

            # Kiểm tra tính nối tiếp: sân bay đến của leg trước == sân bay đi của leg sau
            if prev_leg['ArrivalAirportID'] != next_leg['DepartureAirportID']:
                break  # Dừng nếu không nối tiếp

            current_chain.append(next_leg)

            # Sinh hành trình nếu có ít nhất 2 chặng
            if len(current_chain) >= 2:
                origin = current_chain[0]['DepartureAirportID']
                destination = current_chain[-1]['ArrivalAirportID']
                via = "-".join(str(leg['ArrivalAirportID']) for leg in current_chain[:-1])
                description = f"{origin}-{destination} via {via}"

                yield {
                    'ItineraryID': itinerary_id,
                    'OriginAirportID': origin,
                    'DestinationAirportID': destination,
                    'NumberOfSegments': len(current_chain),
                    'ItineraryDescription': description,
                    'IsTemplate': random.choice([False, True]),
                    'CreatedAt': int(datetime.utcnow().timestamp()),
                    'UpdatedAt': int(datetime.utcnow().timestamp())
                }
                itinerary_id += 1

def stream_generate_itineraries_all_combinations(flight_leg_csv, output_dir="generated_itineraries_full_new", chunk_size=500):
    os.makedirs(output_dir, exist_ok=True)

    flight_leg_df = pd.read_csv(flight_leg_csv)
    generator = generate_all_itineraries(flight_leg_df, max_legs=5)

    buffer = []
    file_index = 1

    for itinerary in generator:
        buffer.append(itinerary)
        if len(buffer) >= chunk_size:
            df = pd.DataFrame(buffer)
            filename = os.path.join(output_dir, f"Itinerary_part_{str(file_index).zfill(3)}.csv")
            df.to_csv(filename, index=False)
            print(f"✅ Ghi {len(buffer)} dòng vào {filename}")
            buffer = []
            file_index += 1

    if buffer:
        df = pd.DataFrame(buffer)
        filename = os.path.join(output_dir, f"Itinerary_part_{str(file_index).zfill(3)}.csv")
        df.to_csv(filename, index=False)
        print(f"✅ Ghi {len(buffer)} dòng cuối vào {filename}")

# 📦 Gọi hàm chính
stream_generate_itineraries_all_combinations("Data/OperationalFlightLeg_merged_001.csv", chunk_size=500)


FileNotFoundError: [Errno 2] No such file or directory: 'Data/OperationalFlightLeg_merged_001.csv'

In [5]:
import pandas as pd

# Load dữ liệu leg
df = pd.read_csv("Data/OperationalFlightLeg_merged_001.csv")

# Danh sách các sân bay theo thứ tự
airport_sequence = [6765, 1105, 10138, 11347]

# Tạo danh sách các cặp chặng cần kiểm tra
segments_to_check = list(zip(airport_sequence[:-1], airport_sequence[1:]))

# Kiểm tra từng chặng
missing_segments = []

for dep, arr in segments_to_check:
    match = df[
        (df["DepartureAirportID"] == dep) &
        (df["ArrivalAirportID"] == arr)
    ]
    if match.empty:
        missing_segments.append((dep, arr))

# In kết quả
if not missing_segments:
    print("✅ Tất cả 4 chặng đều tồn tại trong bảng OperationalFlightLeg.")
else:
    print("❌ Các chặng sau KHÔNG tồn tại:")
    for dep, arr in missing_segments:
        print(f"  - {dep} ➝ {arr}")


❌ Các chặng sau KHÔNG tồn tại:
  - 1105 ➝ 10138
  - 10138 ➝ 11347
